In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label',axis=1)

In [6]:
y = df['label']

In [7]:
import tensorflow as tf

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
vocab_size = 5000

In [11]:
messages = X.copy()
messages.reset_index(inplace = True)

In [13]:
import re 
from nltk.corpus import stopwords

In [14]:
from nltk.stem import WordNetLemmatizer

In [15]:
lemmatizer = WordNetLemmatizer()

In [22]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
onehot_repr = [one_hot(words,vocab_size) for words in corpus]

In [25]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3501 3394 4943]
 [   0    0    0 ... 2766 2697 1517]
 [   0    0    0 ... 2478 4438  352]
 ...
 [   0    0    0 ...  973 3798 2270]
 [   0    0    0 ... 1316 3400 4859]
 [   0    0    0 ... 3365 3731 4583]]


In [28]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size = 0.2,random_state=42)

In [31]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 14628 samples, validate on 3657 samples
Epoch 1/10
14628/14628 [==============================] - 10s 717us/sample - loss: 0.3139 - accuracy: 0.8510 - val_loss: 0.2094 - val_accuracy: 0.9141
Epoch 2/10
14628/14628 [==============================] - 8s 551us/sample - loss: 0.1376 - accuracy: 0.9465 - val_loss: 0.2032 - val_accuracy: 0.9144
Epoch 3/10
14628/14628 [==============================] - 9s 622us/sample - loss: 0.0935 - accuracy: 0.9634 - val_loss: 0.2334 - val_accuracy: 0.9125
Epoch 4/10
14628/14628 [==============================] - 9s 602us/sample - loss: 0.0626 - accuracy: 0.9785 - val_loss: 0.2685 - val_accuracy: 0.9111
Epoch 5/10
14628/14628 [==============================] - 9s 596us/sample - loss: 0.0338 - accuracy: 0.9900 - val_loss: 0.3342 - val_accuracy: 0.9027
Epoch 6/10
14628/14628 [==============================] - 9s 587us/sample - loss: 0.0214 - accuracy: 0.9925 - val_loss: 0.4437 - val_accuracy: 0.9095
Epoch 7/10
14628/14628 [==========================

# Perfomance Metrics and Accuracy

In [32]:
y_pred = model.predict_classes(X_test)

In [33]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [34]:
confusion_matrix(y_test,y_pred)

array([[1895,  187],
       [ 144, 1431]], dtype=int64)

In [35]:
accuracy_score(y_test,y_pred)

0.9094886519004649